## Normalizador de texto

Para otimizar nas analises de textos, pegar um texto e limpar e um otimo processo

In [1]:
import nltk
import re
import unicodedata
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('portuguese')

In [2]:
#Limpar tags html do texto se tiver
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [3]:
#Remover palavraas com acento
def remove_accent(text):
    text = unicodedata.normalize('NFKD', text).encode(
        'ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [4]:
#Remover caracter especial
def remove_special_char(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [5]:
#Remove stopwords
def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    filtered_text = ' '.join(
        [token for token in tokens if token.lower() not in stopword_list])
    return filtered_text

In [6]:
def normalizator(text):

    text = text.lower()

    #remove html
    text = strip_html_tags(text)

    # remove 'stopword'
    text = remove_stopwords(text)

    #remove acento
    text = remove_accent(text)

    #remove new line extra
    text = re.sub(r'[\r|\n|\r\n]+', ' ', text)

    #inserir espaco entre caracter especial
    special_char_pattern = re.compile(r'[\}\}\\\(\)\./!-]')
    text = special_char_pattern.sub(" ", text)

    #remove caracter especial
    text = remove_special_char(text)

    # remove extra whitespace
    text = re.sub(' +', ' ', text)

    return text

# Tf-Idf, Word2Vec e FastText

Alguns exemplos de como se comportam os 3 algoritmos

In [7]:
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
def tfidf():
    vectorizer = TfidfVectorizer()
    return vectorizer

In [9]:
def word2vec(vocab):
    # Criacao do modelo
    model = Word2Vec(vocab, size=200,window=10,min_count=2,workers=10, sample=0)
    # Treinamento do modelo
    model.train(vocab, total_examples=model.corpus_count, epochs=100)
    
    model.save('model_wv.model')
    
    return model

In [10]:
def fasttext(vocab):
    # Criacao do modelo
    model = FastText(size=200,window=10,min_count=2,workers=10, negative=5, min_n=3, max_n=5, sample=1e-5)
    # Criar vocabulario
    model.build_vocab(vocab)
    # Treinamento do moedelo
    model.train(vocab,total_examples=model.corpus_count, epochs=100)
    
    model.save('model_ft.model')
    
    return model

In [11]:
def create_models(setences):
    
    setences = [normalizator(setence) for setence in setences] 
    
    splited_setence = list()
    
    # Tokenizar palavras para o word2vec e o fasttext
    for setence in setences:
        splited_setence.append(setence.split())
      
    # Instancia modelos
    model_tfidf = tfidf()
    model_wv = word2vec(splited_setence)
    model_ft = fasttext(splited_setence)
    
    return model_tfidf,model_wv, model_ft

In [12]:
def initialize_models():
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-12-885ea923475d>, line 3)

# Instancia do DataSet para teste e criacao dos modelos

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('/home/gabriel/Documents/Datasets/wikipedia.txt', delimiter='\t')
df.columns = ['frases']
setences = [setence for setence in df['frases'][:10000]]
df.head()

,frases
0,A Astronomia é uma ciência natural que estuda ...
1,"Ela está preocupada com a evolução , a física ..."
2,A astronomia é uma das mais antigas ciências .
3,Culturas pré - históricas deixaram registrados...
4,"As primeiras civilizações , como os babilônios..."


In [15]:
# Tamanho dos texto

lengths = [len(setence) for setence in df['frases']]
lengths = pd.DataFrame(lengths, columns=["count"])
lengths.describe()

,count
count,7.860086e+06
mean,1.232191e+02
std,1.337409e+02
min,1.000000e+00
25%,6.300000e+01
50%,1.070000e+02
75%,1.640000e+02
max,7.512000e+04


In [16]:
model_tfidf ,model_wv, model_ft = create_models(setences)
print(model_wv, '\n', model_ft, '\n')

/home/gabriel/.local/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Word2Vec(vocab=13482, size=200, alpha=0.025) 
 FastText(vocab=13482, size=200, alpha=0.025) 



# Teste de Similaridade 

Teste com os algoritmos para ver o desempenho dependendo da frase

In [17]:
import string
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# Media dos vetores do FastText e Word2Vec
def avg_feature_vector(sentence, model, num_features, index2word_set, fasttext=True):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    
    if fasttext == True:
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])
            else:
                try:
                    word = model.most_similar(word)[0][0]
                    n_words += 1
                    feature_vec = np.add(feature_vec, model[word])
                except KeyError:
                    continue
    else:
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])                 
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [19]:
# Teste de similaridade entre duas frases Word2Vec e FastText
def similarity(setence1, setence2, model, index2word_set, isfasttext=True):
    s1_avg = avg_feature_vector(normalizator(setence1), model=model, num_features=200, index2word_set=index2word_set, fasttext=isfasttext)
    s2_avg = avg_feature_vector(normalizator(setence2), model=model, num_features=200, index2word_set=index2word_set, fasttext=isfasttext)
    percent = cosine_similarity([s1_avg], [s2_avg])[0][0]
    return "{0:.2}".format(percent) + ' %'

In [48]:
# Similaridade com Word2Vec e FastText
def test_similarity(setence1, setence2):
    
    print("---------- Similarity ----------\n")
    print('Setence 1 =',setence1,' \nSetence 2 =',setence2, '\n')
    
    # Similaridade TfIfd
    tfidf_sim = model_tfidf.fit_transform([normalizator(setence1),normalizator(setence2)])
    print("TfIdf\t\t-> \t",  '{0:.2}'.format(((tfidf_sim * tfidf_sim.T).A)[0,1]) + ' %')
    
    # Similaridade Word2Vec
    index2word_set_wv = model_wv.wv.vocab
    wv_sim = similarity(setence1, setence2, model_wv, index2word_set_wv, isfasttext=False)
    print("Word2Vec\t-> \t", wv_sim)
    
    
    # Similaridade FastText
    index2word_set_ft = model_ft.wv.vocab
    ft_sim = similarity(setence1, setence2, model_ft, index2word_set_ft,isfasttext=True)
    print("FastText\t-> \t", ft_sim) 
    

In [49]:
test_similarity("queria saber como abro uma conta universitaria" , "queria ver o saldo da minha conta!")

---------- Similarity ----------

Setence 1 = queria saber como abro uma conta universitaria  
Setence 2 = queria ver o saldo da minha conta! 

TfIdf		-> 	 0.29 %
Word2Vec	-> 	 0.6 %
FastText	-> 	 0.67 %


/home/gabriel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/gabriel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
